# Numerical Analysis
In this interactive Notebook both the simulation and the deterministic algorithms are assessed on their numerical precision.

In [200]:
# Libraries used in the interactive Notebook
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import random
from tqdm import tqdm

In [201]:
# These are the algorithms, which we are assessing
def simulation(p_is, n):
    amnts = [0]*(len(p_is)+1)
    for _ in range(n):
        goal_amnt = sum([int(random.random() < p_i) for p_i in p_is])
        amnts[goal_amnt] += 1
    return [amnt/n for amnt in amnts]

def dp(p_is):
    n = len(p_is)
    p_k = [1] + [0]*n
    # Iterate over the dp table
    for i in range(n+1):
        for c in range(i, 0, -1):
            inc = p_is[i-1]*p_k[c-1]
            p_k[c-1] -= inc
            p_k[c] += inc
    return p_k

from poibin.poibin import PoiBin

def fft(p_is):
    return PoiBin(p_is).get_pmf_xi()

In [202]:
# Non-float code
def collapse(num):
    num_str = str(num[0])
    exponent_shift = 0
    for c in reversed(num_str):
        if c != '0':
            break
        exponent_shift += 1
    return (int(num_str[:len(num_str)-exponent_shift]), num[1]+exponent_shift)

def floatstring_to_tf(s):
    # Get decimal point index
    point_index = s.find('.')
    decimals_string = '0'
    exponent = 0
    for c in s[point_index+1:]:
        decimals_string += c
        exponent -= 1
    return (int(s[:point_index]+decimals_string), exponent)

def ratio_to_tf(r):
    return collapse(r[0]*r[1], 0)

def scaled_minus(a, b):
    if a[1] == b[1]:
        res = collapse((a[0]-b[0], a[1]))
        return res
    elif a[1] > b[1]:
        exp_shift = a[1]-b[1]
        res = collapse((int(str(a[0]) + '0'*exp_shift) - b[0], b[1]))
        return res
    else:
        print('is_here')
        exp_shift = b[1]-a[1]
        res = collapse((a[0] - int(str(b[0]) + '0'*exp_shift), a[1]))
        return res

def scaled_plus(a, b):
    if a[1] == b[1]:
        return collapse((a[0]+b[0], a[1]))
    elif a[1] > b[1]:
        exp_shift = a[1]-b[1]
        return collapse((int(str(a[0]) + '0'*exp_shift) + b[0], b[1]))
    else:
        exp_shift = b[1]-a[1]
        return collapse((a[0] + int(str(b[0]) + '0'*exp_shift), a[1]))
    
def scaled_mult(a, b):
    return collapse((a[0]*b[0], a[1]+b[1]))

def dp_scaled(p_is):
    n = len(p_is)
    p_k = [(1, 0)] + [(0, 0)]*n
    # Iterate over the dp table
    for i in range(n+1):
        for c in range(i, 0, -1):
            inc = scaled_mult(p_is[i-1], p_k[c-1])
            p_k[c-1] = scaled_minus(p_k[c-1], inc)
            p_k[c] = scaled_plus(p_k[c], inc)
    return p_k

In [203]:
# Read the Possession xG values and group them by match id
xg_df = pd.read_pickle('possession_xGs.pkl')

In [204]:
samples = [float(str(x)) for x in xg_df.sample(11, replace=True)['possession_xg'].to_list()]
samples_transformed = [floatstring_to_tf(str(x)) for x in samples]

In [205]:
# Sampled values rounded and with exponent
print(samples)
poibin_scaled = dp_scaled(samples_transformed)
print(dp(samples))
print(poibin_scaled)


[0.02407022, 0.041848164, 0.05304006, 0.03223021, 0.13323733, 0.02326097, 0.48887327, 0.17002904, 0.062339082, 0.04361772, 0.053362656]
[0.26126893496486403, 0.4350055384591034, 0.23183811544101834, 0.061385056146680134, 0.009505928700490992, 0.0009332624506570753, 6.046752973846429e-05, 2.6198662337466827e-06, 7.506373907545177e-08, 1.3632206856572804e-09, 1.41896270331774e-11, 6.436615222695281e-14]
[(2612689349648640212859722334403123933460034110104255539499615766557782690155748782814756864, -91), (4350055384591034597037323892946382705336272008225086488534677321496215218298912125037674496, -91), (231838115441018349605380429735115713423524234067100610576169904555304951988469569481162752, -90), (61385056146680135782285492614591948589785708244777519986120998952272562188814395556511744, -90), (9505928700490993097158752738384527068512445628478491483707290029429836177925000886976512, -90), (9332624506570752546610520767833856508433041453265274939168500946635689635296899582328832, -91), (60

To